For DYNAMIC Peak count and depolarization start times

v1 changelist: 
- ADDED: CTD90, CTD50, Peak_to_50%DecayTime, Peak_to_90%DecayTime, Amplitude, Upstroke Velocity, Vmax (of upstroke)

TO DO:
- Add Diastolic Calcium read-out (baseline)
- Fix plots for Time (s) to be in 1 sec increments instead of 2.5s
- Add spontaneous peak measurement function (paced at 0.5Hz, expect beat every 2sec, all others should be counted as n spontaneous)

# QUESTIONS?: huynh.trung@mayo.edu


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter1d
from itertools import groupby
from operator import itemgetter
import os
import io
import re


# === Config ===
# Please update this path to your actual file path
excel_path = r"C:\Users\m254292\Downloads\Sup Rep Calbryte Results_ALL.xlsx"
band_factor     = 1.5
upstroke_min    = 0.06

APPLY_FILTERS        = True    # toggle filters
GREY_OUT_UNFILTERED  = True    # grey removed events on plots

xls = pd.ExcelFile(excel_path)

def analyze_roi_signal(time, signal, sample_name, roi_label,
                       apply_filters=True,
                       grey_out_unfiltered=True):
    # --- Prep ---
    signal = pd.to_numeric(signal, errors='coerce').dropna().values
    time   = time[: len(signal)]

    Fmin       = np.min(signal)
    normalized = (signal - Fmin) / Fmin
    smoothed   = gaussian_filter1d(normalized, sigma=1)

    slope_initial = np.gradient(smoothed, time)

    # Photobleaching correction
    flat_thresh = np.percentile(np.abs(slope_initial), 20)
    flat_idxs   = np.where(np.abs(slope_initial) < flat_thresh)[0]
    if flat_idxs.size >= 2:
        bleach_curve = np.interp(time, time[flat_idxs], smoothed[flat_idxs])
        bleach_curve[bleach_curve == 0] = 1
        smoothed = smoothed / bleach_curve

    # Thresholds
    s_max, s_med = np.max(smoothed), np.median(smoothed)
    dr            = s_max - s_med
    rising_thresh = 0.05 * dr
    depol_thresh  = 0.02 * dr

    slope = np.gradient(smoothed, time)

    # Rising-edge groups
    edges  = np.where(slope > rising_thresh)[0]
    groups = []
    for k, g in groupby(enumerate(edges), lambda x: x[0] - x[1]):
        grp = [i for _, i in g]
        if len(grp) >= 5:
            groups.append(grp)

    # Candidate peaks
    peeks = []
    for grp in groups:
        start = grp[0]
        end   = min(grp[-1] + 20, len(smoothed))
        peeks.append(np.argmax(smoothed[start:end]) + start)

    # De-dupe
    filt_peeks = []
    for p in peeks:
        if not filt_peeks or p - filt_peeks[-1] > 40:
            filt_peeks.append(p)

    # Metrics
    rows = []
    for pk in filt_peeks:
        window = range(max(0, pk - 100), pk)
        cand   = [i for i in window if slope[i] < depol_thresh]
        if not cand:
            continue
        start_idx = cand[-1]

        pre       = smoothed[max(0, start_idx - 50): start_idx]
        baseline  = np.min(pre)
        peak_val  = smoothed[pk]
        amp       = peak_val - baseline
        vmax      = np.max(slope[start_idx:pk])


        # CTD90
        lvl90 = baseline + 0.1 * amp
        repol90_idx = next((i for i in range(pk + 1, len(smoothed)) if smoothed[i] <= lvl90), None)
        if repol90_idx is None:
            continue
        repol90_time  = time[repol90_idx]
        repol90_level = smoothed[repol90_idx]
        ctd90         = repol90_time - time[start_idx]
        peak_to_90_decay = repol90_time - time[pk]


        # CTD50
        lvl50 = baseline + 0.5 * amp
        repol50_idx = next((i for i in range(pk + 1, len(smoothed)) if smoothed[i] <= lvl50), None)
        if repol50_idx is None:
            continue
        repol50_time = time[repol50_idx]
        ctd50 = repol50_time - time[start_idx]
        peak_to_50_decay = repol50_time - time[pk]


        ratio50_90 = ctd50 / ctd90 if ctd90 > 0 else np.nan
        upstroke   = time[pk] - time[start_idx]

        rows.append({
            'Depolarization_Start_Time_s': time[start_idx],
            'Peak_Time_s'                : time[pk],
            'Amplitude'                  : amp,
            'CTD50_s'                    : ctd50,
            'CTD90_s'                    : ctd90,
            'Peak_to_50%_Decay_Time_s'    : peak_to_50_decay,
            'Peak_to_90%_Decay_Time_s'    : peak_to_90_decay,
            'Upstroke_Time_s'            : upstroke,
            'Vmax'                       : vmax
        })

    # DataFrame
    df_res_raw = pd.DataFrame(rows).reset_index().rename(columns={'index': '_orig'})
    df_res     = df_res_raw.copy()

    # --- Filters ---
    if apply_filters and not df_res.empty:
        # Upstroke filter
        df_res = df_res[df_res['Upstroke_Time_s'] > upstroke_min]

        # Symmetric APD90 trim
        median_ctd90 = df_res['CTD90_s'].median()
        lower = median_ctd90 / band_factor
        upper = median_ctd90 * band_factor
        df_res = df_res[(df_res['CTD90_s'] >= lower) & (df_res['CTD90_s'] <= upper)]

        df_res = df_res.reset_index(drop=True)

    # Figure out which events were removed (only for plotting, not for summary printing)
    removed = (df_res_raw[~df_res_raw['_orig'].isin(df_res['_orig'])]
               if apply_filters and grey_out_unfiltered else
               pd.DataFrame(columns=df_res_raw.columns))

    expected = [
        'Depolarization_Start_Time_s', 'Peak_Time_s', 'Amplitude',
        'CTD50_s', 'CTD90_s', 'Peak_to_50%_Decay_Time_s', 'Peak_to_90%_Decay_Time_s', 'Upstroke_Time_s',
        'Vmax'
    ]
    df_res = df_res.reindex(columns=['_orig'] + expected)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.plot(time, smoothed, label='Signal')

    # Grey-out removed (if desired)
    if grey_out_unfiltered and not removed.empty:
        for i, t in enumerate(removed['Peak_Time_s']):
            ax.axvline(t, color='grey', linestyle='--', alpha=0.3,
                       label='Removed Peak' if i == 0 else None)
        for i, t in enumerate(removed['Depolarization_Start_Time_s']):
            ax.axvline(t, color='grey', linestyle=':', alpha=0.3,
                       label='Removed Depol Start' if i == 0 else None)

    # Kept events
    for i, t in enumerate(df_res['Peak_Time_s']):
        ax.axvline(t, color='green', linestyle='--', alpha=0.8,
                   label='Peak' if i == 0 else None)
    for i, t in enumerate(df_res['Depolarization_Start_Time_s']):
        ax.axvline(t, color='black', linestyle='--', alpha=0.5,
                   label='Depol Start' if i == 0 else None)


    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("ΔF/Fmin")
    ax.set_title(f"{sample_name} | {roi_label} (bleach-corrected)")
    ax.grid(True)
    fig.tight_layout()

    return sample_name, roi_label, df_res, fig

# === Main ===
all_results = []
plots       = []

for sheet_name in xls.sheet_names:
    raw = xls.parse(sheet_name, header=None, nrows=1)
    if raw.dropna(how='all').empty:
        continue
    sample_name = str(raw.iloc[0, 0])
    if sample_name.lower().endswith('.nd2'):
        sample_name = sample_name[:-4]

    df = xls.parse(sheet_name, header=1)
    if df.dropna(how='all').empty:
        continue
    df.columns = df.columns.astype(str).str.strip()

    time_cols = [c for c in df.columns if 'time' in c.lower()]
    if not time_cols:
        continue
    time = pd.to_numeric(df[time_cols[0]], errors='coerce').values

    roi_cols = [c for c in df.columns if c != time_cols[0] and 'Mono' in c]
    for roi in roi_cols:
        samp, roi_label, res_df, fig = analyze_roi_signal(
            time, df[roi], sample_name, roi,
            apply_filters=APPLY_FILTERS,
            grey_out_unfiltered=GREY_OUT_UNFILTERED
        )

        res_df.insert(0, 'Sample', samp)
        res_df.insert(1, 'ROI', roi_label)

        all_results.append(res_df)
        plots.append((samp, roi_label, fig))

        # === FIX: Display the plot immediately after creation ===
        print(f"Displaying plot for: {samp} | {roi_label}")
        display(fig)
        plt.close(fig) # Prevents the figure from being held in memory and displayed again later



# === Only filtered summary ===
if all_results:
    summary_df = pd.concat(all_results, ignore_index=True)

    # === UPDATED: Sort the summary_df by Sample and numerical ROI ===
    summary_df['ROI_num'] = summary_df['ROI'].str.extract('(\\d+)').astype(int)
    summary_df = summary_df.sort_values(by=['Sample', 'ROI_num']).drop(columns=['ROI_num']).reset_index(drop=True)

    print("Filtered summary:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(summary_df)

    # === Calculate mean and standard deviation values per ROI ===
    mean_summary_df = summary_df.groupby(['Sample', 'ROI'])[[
        'Amplitude',
        'CTD50_s',
        'CTD90_s',
        'Peak_to_50%_Decay_Time_s',
        'Peak_to_90%_Decay_Time_s',
        'Upstroke_Time_s',
        'Vmax'
    ]].agg(['mean', 'std']).reset_index()

    # Flatten the column names after aggregation
    mean_summary_df.columns = ['_'.join(col).strip() for col in mean_summary_df.columns.values]
    mean_summary_df = mean_summary_df.rename(columns={'Sample_': 'Sample', 'ROI_': 'ROI'})


    # === UPDATED: Sort the mean_summary_df by Sample and numerical ROI ===
    mean_summary_df['ROI_num'] = mean_summary_df['ROI'].str.extract('(\\d+)').astype(int)
    mean_summary_df = mean_summary_df.sort_values(by=['Sample', 'ROI_num']).drop(columns=['ROI_num']).reset_index(drop=True)



# === ADDED: Display the DataFrames ===
print("Filtered summary:")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(summary_df)

print("\nMean values per ROI:")
display(mean_summary_df)




In [ ]:
# === Dynamically build out_path using os.path ===
folder, basename = os.path.split(excel_path)
name, ext       = os.path.splitext(basename)
new_name        = f"{name} batch analysis{ext}"
out_path        = os.path.join(folder, new_name)

# === ADDED: Sort the plots list to match the summary sheet order ===
# This will sort the plots first by sample name, then by the numerical ROI number
plots.sort(key=lambda x: (x[0], int(re.search(r'#(\d+)', x[1]).group(1))))


# === Write summary_df, mean_summary_df, and plots into the new workbook ===
with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
    # 1) summary table
    summary_df.to_excel(writer, sheet_name='Summary', index=False)

    # 2) mean values table
    mean_summary_df.to_excel(writer, sheet_name='Mean_Values', index=False)

    # 3) embed plots
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Plots')

    img_row = 0
    for sample, roi, fig in plots:
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)

        worksheet.write(img_row, 0, f"{sample} | {roi}")
        img_row += 1

        worksheet.insert_image(
            img_row, 0,
            f"{sample}_{roi}.png",
            {'image_data': buf, 'x_scale': 0.8, 'y_scale': 0.8}
        )
        img_row += 30

print(f"\n✅ Saved batch analysis to:\n  {out_path}")

To Do: 
- Add APD50
- Add ratio of APD50/90
https://github.com/trunghuynh25/APD-VSC/blob/2fc292274f67dec9940add6c963d6449c9ae6adf/APD%20v6b.ipynb